In [1]:
import pandas as pd
import numpy as np
import json

#cargo los datasets
#si agrego  dtype='unicode' evito algunos errores sobre tipos de datos
dfMovies = pd.read_csv(r'datasets\movies_dataset.csv', dtype='unicode')
dfCredits= pd.read_csv(r'datasets\credits.csv', dtype='unicode')

pd.set_option("display.max_rows", 2000, "display.max_columns", None)

In [32]:
#FUNCIONES

#desanida columna y devuelve un dataframe
def desanidarColumna(columna):
    lista = []
    listaDict=[]
    counter=0
    for k, v in columna.items():
        if isinstance(v, str):
            lista.append(v)
        else:
            lista.append(None)
    for i in lista:   
                try:
                    i=json.loads(lista[counter].replace("'", '"'))
                    listaDict.append(i)
                    counter+=1
                except:
                    None
                    counter+=1
    listaDict=pd.json_normalize(listaDict)
    return listaDict

def desanidarVariasColumnas(columna):
    lista=[]
    for col in columna:            
        #normalizo cada columna
        colAnidada=pd.json_normalize(columna[col])
        lista.append(colAnidada)
        #meto cada columna en una lista

        #armo un df concatenando cada columna de la lista, elimino Nulos y dups
        nuevoDF=pd.concat(lista).dropna()
    return nuevoDF


In [3]:
#desanido belongs to collection
dfPeliculas = (desanidarColumna(dfMovies['belongs_to_collection']))
dfPeliculas


,id,name,poster_path,backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
2,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
3,645.0,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg
4,117693.0,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg
...,...,...,...,...
3166,123720.0,Frankenstein (Hammer Series),/fKiok5TbCKWRgfNd4Rj8gPMzGJp.jpg,/gl9Gi60C9WYVkObOMll19OROhMC.jpg
3167,37261.0,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
3168,37261.0,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
3169,37261.0,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg


In [13]:
#la columna generos hay que desanidarle cada columna despues de aplciarle la funcion

dfGeneros = (desanidarColumna(dfMovies['genres'])) #le aplico la primera funcion
dfGeneros=desanidarVariasColumnas(dfGeneros) #desanido cada columna con la segunda funcion

dfGeneros

#no me queda claro como voy a unir cada genero con su pelicula

#considerar esto mientras sigo revisando el resto del dataframe, mas que todo cuando hago el drop dups

#REVISAR TODAS LAS COLUMNAS PORQUE TIENEN DATOS ADEMAS DE NULOS

,id,name
0,16.0,Animation
1,12.0,Adventure
2,10749.0,Romance
3,35.0,Comedy
5,28.0,Action
12,10751.0,Family
13,36.0,History
15,18.0,Drama
17,80.0,Crime
28,14.0,Fantasy


In [ ]:
#desanido production_companies
#hay que revisar como manejo los duplicados, si esto es solo una tabla foranea tengo que eliminar
#todos los duplicados pero tengo que ver bien como resulta el uso de cada columna

dfCompanies=pd.DataFrame(dfMovies['production_companies'])
dfCompanies=desanidarColumna(dfMovies['production_companies'])
dfCompanies=desanidarVariasColumnas(dfCompanies)
dfCompanies

In [ ]:
#desanido production_countries
#hay que revisar como manejo los duplicados, si esto es solo una tabla foranea tengo que eliminar
#todos los duplicados pero tengo que ver bien como resulta el uso de cada columna

dfProdCountries=desanidarVariasColumnas(desanidarColumna(dfMovies["production_countries"]))
dfProdCountries



In [39]:
#desanido spoken_languages
#hay que revisar como manejo los duplicados, si esto es solo una tabla foranea tengo que eliminar
#todos los duplicados pero tengo que ver bien como resulta el uso de cada columna

dfSpokenLan=desanidarVariasColumnas(desanidarColumna(dfMovies["spoken_languages"]))
dfSpokenLan



,iso_639_1,name
0,en,English
1,en,English
2,en,English
3,en,English
4,en,English
...,...,...
14092,yi,
35269,ja,日本語
14092,zh,普通话
35269,ko,한국어/조선말


In [40]:
dfMovies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6
45464,False,NaN,0,[],NaN,227506,tt